<a href="https://colab.research.google.com/github/jlzxian/Python/blob/master/GPU_calculation_in_python_with_Cupy_and_Numba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cupy as cp

In [ ]:
from numba import cuda

## Getting device information
Additional GPU properties can be found as detailed in this stackoverflow thread: https://stackoverflow.com/questions/62457151/access-gpu-hardware-specifications-in-python. This GPU has 16 GB of RAM.

In [ ]:
cuda.detect()

Found 1 CUDA devices
id 0            b'Tesla K80'                              [SUPPORTED]
                      compute capability: 3.7
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


True

## Sending data to the GPU

In [ ]:
array_cpu = np.random.randint(0, 255, size=(4000, 4000))

In [ ]:
array_cpu.nbytes / 1e6

128.0

In [ ]:
array_gpu = cp.asarray(array_cpu)

In [ ]:
%%timeit

cp.asarray(array_cpu)

10 loops, best of 5: 17.7 ms per loop


In [ ]:
type(array_gpu)

cupy._core.core.ndarray

In [ ]:
from scipy import fft

In [ ]:
%%timeit

fft.fftn(array_cpu)

1 loop, best of 5: 409 ms per loop


In [ ]:
from cupyx.scipy import fft as fft_gpu

In [ ]:
%%timeit

fft_gpu.fftn(array_gpu)

The slowest run took 10648.86 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 125 µs per loop


In [ ]:
fft_cpu = fft.fftn(array_cpu)
fft_sent_back = cp.asnumpy(fft_gpu.fftn(array_gpu))
np.allclose(fft_sent_back, fft_cpu)

True

In [ ]:
# this will fail
fft.fftn(array_gpu)

TypeError: ignored

In [ ]:
# this will also fail
fft_gpu.fftn(array_cpu)

TypeError: ignored

In [ ]:
# some numpy functions may work
np.max(array_gpu)

array(254)

In [ ]:
%%time

# if at all possible, create arrays directly on the GPU
random_gpu_array = cp.random.randint(0, 255, size=(100, 100))

CPU times: user 1.93 s, sys: 25.1 ms, total: 1.95 s
Wall time: 2.15 s


## Custom kernels with numba

### Numba device arrays

In [ ]:
# numba has its own API for transfering data
d_ary = cuda.to_device(array_cpu)
d_ary

In [ ]:
cp.asarray(d_ary)

array([[151,  93,  37, ..., 175,  71, 223],
       [172, 251, 158, ..., 195, 174, 220],
       [ 55, 208, 164, ..., 217, 153, 233],
       ...,
       [252, 195, 231, ..., 177,  49, 121],
       [  4, 123,  72, ...,  60,  35, 250],
       [144, 144, 106, ..., 147,   7, 201]])

In [ ]:
d_ary.copy_to_host()

array([[151,  93,  37, ..., 175,  71, 223],
       [172, 251, 158, ..., 195, 174, 220],
       [ 55, 208, 164, ..., 217, 153, 233],
       ...,
       [252, 195, 231, ..., 177,  49, 121],
       [  4, 123,  72, ...,  60,  35, 250],
       [144, 144, 106, ..., 147,   7, 201]])

### Numba kernels

In [ ]:
from numba import cuda
import cupy as cp
import numpy as np

In [ ]:
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)  
    if i < C.shape[0] and j < C.shape[1]:   # grid can extend beyond C
        tmp = 0.                            
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]        # multiply elements in row i of A and column j of B and add to temp
        C[i, j] = tmp

In [ ]:
cp.random.seed(42)
A = cp.random.uniform(1, 10, size=(2000, 2000), dtype=np.float64)  # array 1
B = cp.random.uniform(1, 10, size=(2000, 2000), dtype=np.float64)  # array 2
C = cp.zeros((2000, 2000), dtype=np.float64)       # array where we store answer 

In [ ]:
C

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
threadsperblock = (16, 16)  # each block will contain 16x16 threads, typically 128 - 512 threads/block
blockspergrid_x = int(np.ceil(C.shape[0] / threadsperblock[0]))
blockspergrid_y = int(np.ceil(C.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)  # we calculate the gridsize (number of blocks) from array
print(blockspergrid)
print(f"The kernel will be executed up to element {threadsperblock[0]*blockspergrid_x}")

(125, 125)
The kernel will be executed up to element 2000


In [ ]:
# execution of the kernel
matmul[blockspergrid, threadsperblock](A, B, C)

In [ ]:
C

array([[59394.46607842, 58001.66377549, 58910.89964126, ...,
        58755.23643036, 59265.65525416, 58447.86197932],
       [59656.82462269, 58635.04995946, 59080.54393462, ...,
        59327.90030958, 60391.24930458, 59425.35827899],
       [62192.77335924, 60700.17680915, 60538.34933653, ...,
        61027.03460329, 61711.10155029, 60544.69882075],
       ...,
       [60649.27416407, 59951.20972379, 60170.2004206 , ...,
        60203.88074659, 60934.19598791, 59613.28418599],
       [61620.11922557, 61264.33868343, 62076.33462258, ...,
        61227.57661876, 62642.97523374, 61841.46799761],
       [61535.95697543, 59600.43760873, 59927.620961  , ...,
        60738.55627077, 61429.70009593, 59662.34901713]])

In [ ]:
# faster multiplication can be obtained by making use of shared memory between threads in the same block
# this requires more thinking about non-obvious implementation!

from numba import float32, int32, float64

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [ ]:
# execution of the kernel
SIZE = 2000
A = cp.random.uniform(1, 10, size=(SIZE, SIZE), dtype=np.float32)
B = cp.random.uniform(1, 10, size=(SIZE, SIZE), dtype=np.float32)
C_slow = cp.zeros((SIZE, SIZE), dtype=np.float32)      
C_fast = cp.zeros((SIZE, SIZE), dtype=np.float32)

In [ ]:
threadsperblock = (TPB, TPB)
blockspergrid = int(np.ceil(SIZE / threadsperblock[0]))
blockspergrid = (blockspergrid, blockspergrid)

In [ ]:
matmul[blockspergrid, threadsperblock](A, B, C_slow)

In [ ]:
fast_matmul[blockspergrid, threadsperblock](A, B, C_fast)

In [ ]:
cp.allclose(C_slow, C_fast)

array(True)

In [ ]:
%%time

for i in range(10):
  matmul[blockspergrid, threadsperblock](A, B, C_slow)

CPU times: user 16.9 ms, sys: 0 ns, total: 16.9 ms
Wall time: 18.1 ms


In [ ]:
%%time

for i in range(10):
  fast_matmul[blockspergrid, threadsperblock](A, B, C_fast)

CPU times: user 17.4 ms, sys: 51 µs, total: 17.4 ms
Wall time: 17.3 ms


In [ ]:
C_c = cp.dot(A, B)

In [ ]:
np.allclose(C_c, C_fast)

array(True)

In [ ]:
%time

for i in range(10):
  cp.dot(A, B)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [ ]:
del 